# Convolutional Neural Network

In [1]:
import numpy as np
import pandas as pd
from matplotlib import style 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from scipy import signal

style.use("ggplot")

%matplotlib inline

In [2]:
data_folder = "./data/shapes/"

shapes_classes = os.listdir(data_folder)
shapes_classes

['square', 'circle', 'triangle']

In [3]:
import tensorflow
from tensorflow import keras

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
image_size = (128,128)
batch_size = 30

datagen = ImageDataGenerator(rescale=1.0/255,validation_split=0.3)

In [6]:
training_generator = datagen.flow_from_directory(
    directory=data_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

testing_generator = datagen.flow_from_directory(
    directory=data_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 840 images belonging to 3 classes.
Found 360 images belonging to 3 classes.


In [7]:
class_labels = training_generator.class_indices.keys()

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    MaxPooling2D,
    Dropout,
    Flatten
)

In [10]:
model= Sequential()

## Convolution 


[Understanding Convolution](https://nishantbaheti.github.io/machineLearningExploration/MathExploration/Convolution.html#Understanding-Convolution)

In [12]:
# The input layer accepts an image and applies a convolution that uses 32 6x6 filters and a rectified linear unit activation function
model.add(
    Conv2D(
        filters=32, 
        kernel_size=(6, 6), 
        input_shape=training_generator.image_shape, 
        activation='relu'
    )
)

## Pooling

- After extracting feature values from images, pooling (or downsampling) layers are used to reduce the number of feature values while retaining the key differentiating features that have been extracted.
- The feature map extracted by a filter in a convolutional layer contains an array of feature values.
- A pooling kernel is used to reduce the number of feature values. In this case, the kernel size is 2x2, so it will produce an array with quarter the number of feature values.
- The pooling kernel is convolved across the feature map, retaining only the highest pixel value in each position.


        0   0   0                             155 255
        0  155 255  =>  apply 2X2 pooling =>  155 255
        155 0   0

In [13]:

# Next we'll add a max pooling layer with a 2x2 patch
model.add(
    MaxPooling2D(
        pool_size=(2,2)
    )
)

- We can add as many layers as we think necessary - here we'll add another convolution and max pooling layer

In [14]:
model.add(
    Conv2D(
        filters=32, 
        kernel_size=(6, 6), 
        activation='relu'
    )
)
model.add(
    MaxPooling2D(
        pool_size=(2, 2)
    )
)

# And another set
model.add(
    Conv2D(
        filters=32, 
        kernel_size=(6, 6), 
        activation='relu'
    )
)
model.add(
    MaxPooling2D(
        pool_size=(2, 2)
    )
)

## Dropping layer

One of the most difficult challenges in a CNN is the avoidance of overfitting, where the resulting model performs well with the training data but doesn't generalize well to new data on which it wasn't trained. One technique you can use to mitigate overfitting is to include layers in which the training process randomly eliminates (or "drops") feature maps. This may seem counterintuitive, but it's an effective way to ensure that the model doesn't learn to be over-dependent on the training images.

Other techniques you can use to mitigate overfitting include randomly flipping, mirroring, or skewing the training images to generate data that varies between training epochs.

In [15]:
# A dropout layer randomly drops some nodes to reduce inter-dependencies (which can cause over-fitting)
model.add(
    Dropout(0.2)
)

## Flattening layer

After using convolutional and pooling layers to extract the salient features in the images, the resulting feature maps are multidimensional arrays of pixel values. A flattening layer is used to flatten the feature maps into a vector of values that can be used as input to a fully connected layer.

In [16]:
# Flatten the feature maps 
model.add(Flatten())

## Generate a fully-connected output layer with a predicted probability for each class

In [17]:
# (softmax ensures all probabilities sum to 1)
model.add(
    Dense(
        training_generator.num_classes, 
        activation='softmax'
    )
)

# With the layers defined, we can now compile the model for categorical (multi-class) classification
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      3488      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        36896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 32)        36896     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 32)        0

In [20]:
# Train the model over 5 epochs using 30-image batches and using the validation holdout dataset for validation
num_epochs = 5
history = model.fit(
    training_generator,
    steps_per_epoch = training_generator.samples // batch_size,
    validation_data = testing_generator, 
    validation_steps = testing_generator.samples // batch_size,
    epochs = num_epochs)

Epoch 1/5
28/28 [==============================] - 53s 2s/step - loss: 0.7798 - accuracy: 0.6464 - val_loss: 0.4189 - val_accuracy: 0.8250
Epoch 2/5
28/28 [==============================] - 54s 2s/step - loss: 0.2636 - accuracy: 0.8905 - val_loss: 0.1260 - val_accuracy: 0.9778
Epoch 3/5
28/28 [==============================] - 50s 2s/step - loss: 0.0759 - accuracy: 0.9821 - val_loss: 0.0149 - val_accuracy: 0.9972
Epoch 4/5
28/28 [==============================] - 46s 2s/step - loss: 0.0199 - accuracy: 0.9964 - val_loss: 0.0262 - val_accuracy: 0.9889
Epoch 5/5
28/28 [==============================] - 45s 2s/step - loss: 0.0096 - accuracy: 0.9976 - val_loss: 0.0086 - val_accuracy: 1.0000


In [22]:
history.history

{'loss': [0.7798408269882202,
  0.2636471390724182,
  0.07594814151525497,
  0.01989593915641308,
  0.009630522690713406],
 'accuracy': [0.6464285850524902,
  0.8904761672019958,
  0.9821428656578064,
  0.9964285492897034,
  0.9976190328598022],
 'val_loss': [0.41894739866256714,
  0.12595893442630768,
  0.014909405261278152,
  0.026197640225291252,
  0.008617321960628033],
 'val_accuracy': [0.824999988079071,
  0.9777777791023254,
  0.9972222447395325,
  0.9888888597488403,
  1.0]}